In [ ]:
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sn
from sklearn import metrics
import time
# Các bước thực hiện
# 1. Đọc file tests training & label
# 2. Chia 3, lấy phần giữa (phần ổn định)
# 3. Phân khung 30ms, dịch khung 20ms, khung chữ nhật
# 4. Tính MFCC trên từng khung => trung bình MFCC trên tất cả các khung => đặc trưng của 1 audio
# 5. Tính trung bình đặc trưng của 21 audio của mỗi nguyên âm => đặc trưng nguyên âm
#### Lưu lại các vector hệ số
# 6. Đọc file input test
# 7. Chia 3 lấy phần giữa
# 8. Phân khung 30ms, dịch khung 20ms, khung chữ nhật

frame_length = 0.03
frame_hop = 0.02
K = 5  # 2 3 4 5

# functions ---------------------
def get_stable_audio(filepath):  # trả về phần ổn định ở chính giữa của audio
    audio, sr = librosa.load(filepath, sr=None)
    div_part = len(audio) // 3
    center_part = audio[div_part : 2 * div_part]
    center_part = center_part/(max(abs(center_part)))
    return center_part, sr


def windowing(
    audio, sr, frame_length=0.03, frame_hop=0.025
):  # windowing với length, hop
    frame_size = int(frame_length * sr)
    hop_size = int(frame_hop * sr)
    
    frames = []
    for i in range(0, len(audio), hop_size):
        if i + frame_size < len(audio):
            frames.append(audio[i : i + frame_size])
    return frames


def mfcc(frame, N, sr):  # tính N đặc trưng MFCC trên frame, viết gọn của librosa
    return librosa.feature.mfcc(y=frame, sr=sr, n_mfcc=N)


# Đọc input
test_path = ".\\trimmed_signals\\HuanLuyen-16k"

input = []
N_MFCC = 13
for file in sorted(os.listdir(test_path)):
    # print(os.path.join(path,file))
    sub_path = os.path.join(test_path, file)
    for sub_file in sorted(os.listdir(sub_path)):
        exact_test_path = os.path.join(sub_path, sub_file)
        label = sub_file.split(".")[0]

        audio, sr = get_stable_audio(exact_test_path)
        frames = windowing(audio, sr, frame_length, frame_hop)

        # Tính trung bình trước rồi gom nhóm
        MFCCs = []
        for frame in frames:
            MFCCs.append(mfcc(frame,N_MFCC,sr))
        feature = np.average(MFCCs,axis=0)
        feature = feature/max(abs(feature))

        input.append([feature, label])

        # Gom nhóm không cần tính trung bình theo từng audio
        # for frame in frames:
        #     vector = mfcc(frame, N_MFCC, sr)
        #     vector = vector/max(abs(vector))
        #     input.append([vector, label])


In [ ]:
# grouping by labels
features = {'a': [], 'e':[], 'i':[], 'o':[], 'u':[]}
for j in input:
    features[j[1]].append(j[0])
# print(features['a'])
# print(len(features['a']))

In [ ]:
average_features = {
    "a": np.average(features["a"], axis=0),
    "e": np.average(features["e"], axis=0),
    "i": np.average(features["i"], axis=0),
    "o": np.average(features["o"], axis=0),
    "u": np.average(features["u"], axis=0),
}
# print(average_features['a'])
rows = []
for key, f in average_features.items():
    rows.append([key, f.reshape((N_MFCC,))])
plt.title("Vector trung bình đặc trưng MFCC của 5 nguyên âm")
plt.plot(average_features["a"])
plt.plot(average_features["e"])
plt.plot(average_features["i"])
plt.plot(average_features["o"])
plt.plot(average_features["u"])
plt.legend(["a", "e", "i", "o", "u"])
# plt.savefig("MFCC_Features_Vector.png")
df = pd.DataFrame(rows)
df.to_csv("average_features_MFCC_13.csv", index=False)

In [ ]:
#cài đặt Kmean lên tập features của từng nguyên âm, lấy tập centroids làm tập đặc trưng cho nguyên âm đó

#chọn ngẫu nhiên K hàng trong X làm tâm
def init_K(X, K):
    return X[np.random.choice(X.shape[0], K, replace=False)]

#duyệt tất cả các record và label tâm
def clustering(X, centroids):
    N = X.shape[0]
    label = np.zeros((1,N))
    for i in range (N):
        xi = X[i,:]
        min = np.linalg.norm(xi - centroids[0])
        for index, c in enumerate(centroids):
            if np.linalg.norm(xi - c) < min:
                min = np.linalg.norm(xi - c)
                label[0,i] = index
    return label

def cluster_of_same_label(index, label, X):
    selected = []
    for i in range(label.shape[1]):
        if label[0,i] == index:
            selected.append(X[i])
    return np.array(selected)

def isEqual(last_label, label):
    for i in range (label.shape[1]):
        if last_label[0,i] != label[0,i]:
            return False
    return True

def K_means(X, K, loop = 20):
    count = 0
    centroids = init_K(X,K)
    last_label = np.zeros((1,X.shape[0]))
    while (count < loop):
        label = clustering(X, centroids)
        #kiểm tra, nếu tập label không đổi => thoát
        if isEqual(last_label, label):
            return centroids, label
        new_centroids = np.zeros(centroids.shape)
        for index in range (centroids.shape[0]):
            new_centroids[index] = np.mean(cluster_of_same_label(index,label,X), axis = 0)
        centroids = new_centroids.copy()
        last_label = label
        count+=1
    return centroids, label


dim = N_MFCC
Kmeans_features = {
    'a': K_means(np.array(features['a']),K)[0], 
    'e': K_means(np.array(features['e']),K)[0],
    'i': K_means(np.array(features['i']),K)[0], 
    'o': K_means(np.array(features['o']),K)[0], 
    'u': K_means(np.array(features['u']),K)[0]
    }
# print(Kmeans_features['a'])
# print(Kmeans_features['a'].shape)

In [ ]:
#đọc dữ liệu kiểm thử
test_path = ".\\trimmed_signals\\KiemThu-16k"

tests = []

for file in sorted(os.listdir(test_path)):
    # print(os.path.join(path,file))
        sub_path = os.path.join(test_path,file)
        for sub_file in sorted(os.listdir(sub_path)):
            exact_test_path = os.path.join(sub_path,sub_file)
            audio, sr = get_stable_audio(exact_test_path)
            label = sub_file.split(".")[0]
            frames = windowing(audio, sr, frame_length, frame_hop)
            MFCCs = []
            for frame in frames:
                MFCCs.append(mfcc(frame,N_MFCC,sr))
            feature = np.average(MFCCs,axis=0)
            feature = feature/max(abs(feature))
            tests.append([feature,label]) # label này sẽ được sử dụng để kiểm tra kết quả nhận dạng

# tests = np.array(tests)
# print(tests)

In [ ]:
def classify(test_vectors, features):
    label_result = ""
    min_dist = 1e6
    for key, feature_set in features.items():
        for f in feature_set:
            dist = np.linalg.norm(test_vectors - f)
            if dist < min_dist:
                min_dist = dist
                label_result = key
    return label_result


count = 0
keys = {"a": 0, "e": 1, "i": 2, "o": 3, "u": 4}
table = np.zeros((5, 5))
startTime = time.time()
for test in tests:
    label_result = classify(test[0], Kmeans_features)
    if label_result == test[1]:
        count += 1
    table[keys[label_result], keys[test[1]]] += 1
    # print("Label: " + str(test[1]) + ", classified label: " + str(label_result))
endTime = time.time()
print("Thời gian nhận dạng", (endTime - startTime), "s")

In [ ]:
print(count)
Data = np.array([[K, frame_length, frame_hop, count, str(count/105*100) + "%"]])
Data = np.array(Data)
Data = Data.reshape(1, 5)
print(Data.shape)
# dfData = pd.DataFrame(Data, columns=["K", "frame_length", "frame_hop", "Count", "Percentage"], index=None)
dfData = pd.DataFrame(Data,columns=None ,index=None)
# dfData.to_csv("Test.csv", mode="a", index=False, header=False)
print(dfData)
# hiển thị bảng đúng sai
# từng hàng là label do chương trình nhận dạng
# từng cột là label đúng của dữ liệu
df_cm = pd.DataFrame(np.transpose(table), index=[i for i in "AEIOU"], columns=[i for i in "AEIOU"])
sn.set(font_scale=1)
sn.heatmap(df_cm, annot=True, annot_kws={"size": 10}, cbar=False)
plt.xlabel("predict")
plt.ylabel("test")
# plt.savefig("ConfusionMatrix_K =" + str(K) + ".jpg")
plt.show()